<a href="https://colab.research.google.com/github/tnc-br/ddf-isoscapes/blob/eval_test_results/validation_pipeline_ordinary_krig_random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Validation Pipeline

In [ ]:
DEBUG = False #@param {type:"boolean"}
GDRIVE_BASE = "/content/gdrive" #@param

ISOSCAPE_OXYGEN_MEANS_FILENAME = "canonical/kriging_random_means.tiff" #@param
ISOSCAPE_OXYGEN_VARS_FILENAME = "canonical/kriging_random_vars.tiff" #@param
ISOSCAPE_CARBON_FILENAME = "iso_d13C_map_wood_stack.tiff" #@param
ISOSCAPE_NITROGEN_MEANS_FILENAME = "Raster_Brasil_krig_d15N.tiff" #@param
ISOSCAPE_NITROGEN_VARS_FILENAME = "Brasil_Raster_Krig_SD_d15N.tiff" #@param

TEST_SET_FILENAME = 'canonical/uc_davis_test_fixed_grouped.csv' #@param
ORIGINAL_SET_FILENAME = '2023_06_23_Results_Google.csv' #@param
# Columns of values to read ground truths from. Invalid values are 'truth'
# and 'prediction'.
MEAN_TRUTH_NAME = 'd18O_cel_mean' #@param
VAR_TRUTH_NAME = 'd18O_cel_variance' #@param
# Columns of values to write temporary predictions to (for RMSE calculation).
# Invalid values are 'truth' and 'prediction'.
MEAN_PREDICTED_NAME = 'd18O_predicted_mean' #@param
VAR_PREDICTED_NAME = 'd18O_predicted_variance' #@param

In [ ]:
# Access data stored on Google Drive
if GDRIVE_BASE:
    from google.colab import drive
    drive.mount(GDRIVE_BASE)

if DEBUG:
    %pip install -Uqq ipdb
    import ipdb
    %pdb on

# Import

In [ ]:
import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
#ddfimport.ddf_source_control_pane()
ddfimport.ddf_import_common()

In [ ]:
import importlib
import raster
import hypothesis
import dataset
importlib.reload(raster)
importlib.reload(hypothesis)
importlib.reload(dataset)

# Isoscape: Calculate RMSE for Oxygen



In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import dataset

In [ ]:
# Required to both import raster and read GDrive files
raster.RASTER_BASE = "/MyDrive/amazon_rainforest_files/amazon_rasters/" #@param
raster.SAMPLE_DATA_BASE = "/MyDrive/amazon_rainforest_files/amazon_sample_data/" #@param
raster.TEST_DATA_BASE = "/MyDrive/amazon_rainforest_files/amazon_test_data/" #@param
raster.ANIMATIONS_BASE = "/MyDrive/amazon_rainforest_files/amazon_animations/" #@param
raster.GDRIVE_BASE = "/content/gdrive" #@param

In [ ]:
oxygen_means_isoscape = raster.load_raster(raster.get_raster_path(ISOSCAPE_OXYGEN_MEANS_FILENAME), use_only_band_index=0)
oxygen_vars_isoscape = raster.load_raster(raster.get_raster_path(ISOSCAPE_OXYGEN_VARS_FILENAME), use_only_band_index=0)

In [ ]:
eval_dataset = pd.read_csv(raster.get_sample_db_path(TEST_SET_FILENAME), index_col=0)
eval_dataset.head()

In [ ]:
import evaluation

In [ ]:
mean_rmse, var_rmse, overall_rmse = evaluation.calculate_rmse(
    eval_dataset,
    oxygen_means_isoscape,
    oxygen_vars_isoscape,
    MEAN_TRUTH_NAME, VAR_TRUTH_NAME, MEAN_PREDICTED_NAME, VAR_PREDICTED_NAME)

In [ ]:
print("RMSE of Means:", mean_rmse)
print("RMSE of Vars:", var_rmse)
print("Overall RMSE:", overall_rmse)

# Fraud Detection Hypothesis Test
Combines the p values of each element specified below and classifies as fraudulent with the resulting p-value.

Creating fraudulent samples

In [ ]:
eval_dataset.shape

In [ ]:
elements = ['d18O_cel', 'd15N_wood', 'd13C_wood']
isotope_column_names = ['d18O_cel', 'd15N_wood', 'd13C_wood']
mean_isoscapes = [
    oxygen_means_isoscape,
    raster.load_raster(
        raster.get_raster_path(ISOSCAPE_NITROGEN_MEANS_FILENAME), use_only_band_index=0),
    raster.load_raster(
        raster.get_raster_path(ISOSCAPE_CARBON_FILENAME), use_only_band_index=0),
]
vars_isoscapes = [
    oxygen_vars_isoscape,
    raster.load_raster(
        raster.get_raster_path(ISOSCAPE_NITROGEN_VARS_FILENAME), use_only_band_index=0),
    raster.load_raster(
        raster.get_raster_path(ISOSCAPE_CARBON_FILENAME), use_only_band_index=1),
]

In [ ]:
# Parameters for creation of fraudulent samples.
max_trusted_radius = 0.1
max_fraud_radius = 25
min_fraud_radius = 1

In [ ]:
real_samples_data = pd.merge(eval_dataset[['Code','lat','long',MEAN_TRUTH_NAME, VAR_TRUTH_NAME]],
         pd.read_csv(raster.get_sample_db_path(ORIGINAL_SET_FILENAME), index_col=0), how="inner",
                    left_on=['Code', 'lat', 'long'], right_on=['Code', 'lat', 'long'])

In [ ]:
fake_sample = dataset.create_fraudulent_samples(
    real_samples_data, mean_isoscapes, elements, max_trusted_radius, max_fraud_radius, min_fraud_radius
)

Combine fraudulent and real samples in a Dataframe, identified by 'fraud' column

In [ ]:
real = real_samples_data[['Code','lat','long'] + elements]
real = real.assign(fraud=False)

In [ ]:
test_dataset = pd.concat([real, fake_sample], axis=1, join='outer')
test_dataset = real.append(fake_sample, ignore_index=True)

Calling t-test function that expects the Dataframe with the fraud column

In [ ]:
sample_size_per_location = 5
p_value_target = 0.05

In [ ]:
assert((
  hypothesis.fraud_metrics(test_dataset,
                [isotope_column_names[0]],
                [mean_isoscapes[0]],
                [vars_isoscapes[0]],
                sample_size_per_location,
                0.0)
).accuracy == 0.5)
assert((
  hypothesis.fraud_metrics(test_dataset,
                [isotope_column_names[0]],
                [mean_isoscapes[0]],
                [vars_isoscapes[0]],
                sample_size_per_location,
                1.0)
).accuracy == 0.5)

In [ ]:
assert(test_dataset[test_dataset['fraud'] == True].shape[0] == test_dataset[test_dataset['fraud'] == False].shape[0])

In [ ]:
import numpy as np

p_values = np.linspace(0, 1, 1000)

accuracies = []
for p_value_target in p_values:
  fraud_metrics = (
    hypothesis.fraud_metrics(test_dataset,
                  isotope_column_names,
                  mean_isoscapes,
                  vars_isoscapes,
                  sample_size_per_location,
                  p_value_target)
  )
  accuracies.append(fraud_metrics.accuracy)

In [ ]:
# Accuracy Graph
import matplotlib.pyplot as plt

plt.plot(p_values, accuracies)
plt.xlabel("p-value")
plt.ylabel("Accuracy")
plt.title("Accuracy Graph")
plt.show()

In [ ]:
import numpy as np

predictions = hypothesis.get_predictions(test_dataset,
  isotope_column_names,
  mean_isoscapes,
  vars_isoscapes,
  sample_size_per_location)

In [ ]:
y_true = predictions['fraud']
# Fraud p value is lower the more positive a prediction/label is.
# Inverting it gives us the probability of positive label class (fraud).
y_pred = 1 - predictions['fraud_p_value']

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import auc

precision, recall, thresholds = precision_recall_curve(y_true, y_pred)

plt.plot(recall, precision, label="model")
print(auc(recall, precision))
plt.plot(np.linspace(0, 1, 100), np.ones(100) * 0.5, "--", label="random")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall curve")
plt.legend()
plt.show()